In [2]:
import pandas as pd
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

from IPython.display import display
show = display

from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import KFold
import sklearn.metrics as skm

lda, liwc, mtf = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
train = pd.DataFrame()

# TODO add city as feature
for city, city_train in [("NewYork", "NY"), ("London", "London"), ("Singapore", "Singapore")]:
    lda = pd.concat([lda, pd.read_csv(f"data/Features/features{city}/Twitter/LDA50Features.csv", encoding="ISO-8859-15")], axis=0, ignore_index=True)
    liwc = pd.concat([liwc, pd.read_csv(f"data/Features/features{city}/Twitter/LIWCFeatures.csv", encoding="ISO-8859-15")], axis=0, ignore_index=True)
    mtf = pd.concat([mtf, pd.read_csv(f"data/Features/features{city}/Twitter/manuallyDefinedTextFeatures.csv", encoding="ISO-8859-15")], axis=0, ignore_index=True)

    train = pd.concat([train, pd.read_csv(f"data/{city_train}Train.csv")], axis=0, ignore_index=True)

lda = lda.drop_duplicates(subset=["_id"], ignore_index=True)
liwc = liwc.drop_duplicates(subset=["_id"], ignore_index=True)
mtf = mtf.drop_duplicates(subset=["_id"], ignore_index=True)

# Gender prediction

df = lda.merge(liwc, on='_id').merge(mtf, on='_id')
df = df.merge(train, left_on='_id', right_on='row ID')
# show(df)
# show(df.describe())
# show(df.nunique())
show(df.shape)
show(df.info())
# show(df.isnull().sum())

df = df.dropna(subset=["gender"])

Y = df["gender"].values
X = df.drop([*train.columns, "_id"], axis=1).values

# show(X)
# show(Y)

## not needed now
# df = SimpleImputer(missing_values = 'NaN', strategy = 'mean').fit_transform(df) 
# df = mtf

from sklearn.ensemble import RandomForestClassifier
import joblib

with joblib.parallel_backend("multiprocessing"): 

#     for N in [3, 5, 10, 20, 30, 50, 100, 300, 500]:
#         for D in [3, 5, 10, 20, 30, 50, 100, 300, 500]:
#             show(f"N={N}, D={D}")
    fscore, accuracy = [], []
    fold10 = KFold(n_splits=10, shuffle=True, random_state=42)
    for train_ix, test_ix in fold10.split(X, Y):
        clf = RandomForestClassifier(n_estimators=30, max_depth=300, min_samples_split=2, random_state=42)
#         clf.fit(X[train_ix], Y[train_ix])
        y_true, y_pred = Y[test_ix], clf.predict(X[test_ix])
#         show(y_true)
#         show(y_pred)
        accuracy.append(skm.accuracy_score(y_true, y_pred))
        fscore.append(skm.f1_score(y_true, y_pred, average="weighted"))

    accuracy = np.array(accuracy).mean(axis=0)
    fscore = np.array(fscore).mean(axis=0)
    show(f"accuracy: {accuracy}, f1score: {fscore}")




(9463, 145)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9463 entries, 0 to 9462
Columns: 145 entries, _id to income
dtypes: float64(134), int64(1), object(10)
memory usage: 10.5+ MB


None

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

# Late Fusion

In [6]:
lda = None
lda50, liwc, mtf, image_concepts, lda6 = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
train = pd.DataFrame()

for city, city_train in [("NewYork", "NY"), ("London", "London"), ("Singapore", "Singapore")]:
    lda50 = pd.concat([lda50, pd.read_csv(f"data/Features/features{city}/Twitter/LDA50Features.csv", encoding="ISO-8859-15")], axis=0, ignore_index=True)
#     liwc = pd.concat([liwc, pd.read_csv(f"data/Features/features{city}/Twitter/LIWCFeatures.csv", encoding="ISO-8859-15")], axis=0, ignore_index=True)
#     mtf = pd.concat([mtf, pd.read_csv(f"data/Features/features{city}/Twitter/manuallyDefinedTextFeatures.csv", encoding="ISO-8859-15")], axis=0, ignore_index=True)
    image_concepts = pd.concat([image_concepts, pd.read_csv(f"data/Features/features{city}/Instagram/imageConceptsFeatures.csv", encoding="ISO-8859-15")], axis=0, ignore_index=True)
    lda6 = pd.concat([lda6, pd.read_csv(f"data/Features/features{city}/Foursquare/venueCategoriesLDA6Features.csv", encoding="ISO-8859-15")], axis=0, ignore_index=True)

    train = pd.concat([train, pd.read_csv(f"data/{city_train}Train.csv")], axis=0, ignore_index=True)

lda50 = lda50.drop_duplicates(subset=["_id"], ignore_index=True)
# liwc = liwc.drop_duplicates(subset=["_id"], ignore_index=True)
# mtf = mtf.drop_duplicates(subset=["_id"], ignore_index=True)
image_concepts = image_concepts.drop_duplicates(subset=["_id"], ignore_index=True)
train = train.drop_duplicates(subset=["row ID"], ignore_index=True)
lda6 = lda6.drop_duplicates(subset=["_id"], ignore_index=True)

# show(train)
df = train.merge(lda50, right_on='_id', left_on='row ID', how='left').drop(["_id"], axis=1)
df = df.merge(image_concepts, right_on='_id', left_on='row ID', how='left').drop(["_id"], axis=1)
df = df.merge(lda6, right_on='_id', left_on='row ID', how='left')#.drop(["_id"], axis=1)
df = df.dropna(subset=["gender"])
# show(df)

def prediction_df(clf, df, feature, name):
    df = df.dropna(subset=feature.columns, how="any")
    p = clf.predict_proba(df[feature.columns].drop("_id", axis=1))
    p = pd.DataFrame(p, columns=[f"{name}_{clf.classes_[0]}", f"{name}_{clf.classes_[1]}"])
    p["_id"] = df["_id"]
    return p

# with joblib.parallel_backend("multiprocessing"): 
fscore, accuracy = [], []
fold10 = KFold(n_splits=2, shuffle=True, random_state=42)
for train_ix, test_ix in fold10.split(df.values):
    new_feature = []
    _df = df.drop('row ID', axis=1).copy()
    train_df = df.iloc[train_ix]

    # Twitter classifier

    XY = train_df.dropna(subset=[*lda50.columns], how='any')
    X = XY[lda50.columns].drop(["_id"], axis=1).values
    Y = XY["gender"].values

    clf = RandomForestClassifier(n_estimators=30, max_depth=300, min_samples_split=2, random_state=42)
    clf.fit(X, Y)

    p = prediction_df(clf, df, lda50, "lda50")
    _df = _df.head(20)
    p = p.head(20)
    show(_df)
    show(p)
    _df = _df.merge(p, left_on="_id", right_on="_id", how="left")
#     new_feature += list(p.drop(["_id"], axis=1).columns)

    show(_df)

    # Instagram classifier

#         df = image_concepts.merge(train, left_on='_id', right_on='row ID')
#         df = df.dropna(subset=["gender"])
#         Y = df["gender"].values
#         X = df.drop([*train.columns, "_id"], axis=1).values

#         clf = RandomForestClassifier(n_estimators=30, max_depth=300, min_samples_split=2, random_state=42)
#         clf.fit(X[train_ix], Y[train_ix])

#         # Foursquare classifier

#         df = lda6.merge(train, left_on='_id', right_on='row ID')
#         df = df.dropna(subset=["gender"])
#         Y = df["gender"].values
#         X = df.drop([*train.columns, "_id"], axis=1).values

#         clf = AdaBoostClassifier(n_estimators=50, random_state=42)
#         clf.fit(X[train_ix], Y[train_ix])


#         show(df)




,educationInfoForAgeGroupEstimation,workInfoForAgeGroupEstimation,gender,realAge,ageGroup,relationship,educationLevel,occupation,income,topic_0,...,i998,i999,i1000,_id,ftopic_0,ftopic_1,ftopic_2,ftopic_3,ftopic_4,ftopic_5
0,Universidad Santo Tomï¿½s Class of 2009 ï¿½ Ps...,GMO Chile Analista de Selecciï¿½n y Desarrollo...,female,NaN,NaN,NaN,NaN,NaN,NaN,0.001808,...,NaN,NaN,NaN,062bbc1961e4b64f2c31b85d382b2384,0.188927,0.030995,0.034959,0.080224,0.586303,0.078591
1,NaN,NaN,male,NaN,NaN,NaN,NaN,NaN,NaN,0.000015,...,2.600426,2.36398,2.53352,aeb08521a61fc8e5900bc8dff0d4ba5b,0.062030,0.000452,0.219077,0.041921,0.012263,0.664257
2,NaN,NaN,male,NaN,NaN,in a relationship,NaN,NaN,NaN,0.001001,...,NaN,NaN,NaN,b22e8cc09aeb634daff8a0713dfd1808,0.798687,0.020560,0.023189,0.053214,0.052219,0.052131
3,NaN,Commonwealth of Massachusetts Legislative Aide...,male,NaN,NaN,NaN,NaN,healthcare support,NaN,0.001594,...,NaN,NaN,NaN,1a82db6eb37170043972f0729abf6019,0.696505,0.030995,0.034959,0.080224,0.078724,0.078591
4,NaN,Mindshare Pan Regional Director ï¿½ 1 October ...,female,NaN,NaN,NaN,NaN,management,NaN,0.001808,...,NaN,NaN,NaN,12ff3219501d22c045a8ad0caaabb728,0.188927,0.030995,0.034959,0.080224,0.078724,0.586170
5,West Milford Township High School West Milford...,Jackpocket CEO &amp; Founder ï¿½ May 2013 to p...,male,NaN,NaN,NaN,school,NaN,NaN,0.001808,...,NaN,NaN,NaN,9a071773d2b55f6823530820584a3fba,0.188927,0.538574,0.034959,0.080224,0.078724,0.078591
6,NaN,NaN,male,NaN,NaN,NaN,NaN,NaN,NaN,0.002089,...,NaN,NaN,NaN,981040a6959e0cf593c460210a820707,0.696505,0.030995,0.034959,0.080224,0.078724,0.078591
7,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,NaN,0.001425,...,NaN,NaN,NaN,8ad553eacac5508054181c8b3f70c12b,0.188927,0.030995,0.034959,0.334014,0.078724,0.332381
8,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,NaN,0.002473,...,NaN,NaN,NaN,4eda130f2da931d40253960e88ae5a33,0.696505,0.030995,0.034959,0.080224,0.078724,0.078591
9,NaN,Commonwealth Foundation Executive Director ï¿½...,male,NaN,NaN,NaN,NaN,management,NaN,0.001594,...,NaN,NaN,NaN,a3f0c091a05bfb15a78bde8249660152,0.188927,0.030995,0.034959,0.080224,0.078724,0.586170


,lda50_female,lda50_male,_id
0,0.700000,0.300000,062bbc1961e4b64f2c31b85d382b2384
1,0.100000,0.900000,aeb08521a61fc8e5900bc8dff0d4ba5b
2,0.266667,0.733333,b22e8cc09aeb634daff8a0713dfd1808
3,0.700000,0.300000,1a82db6eb37170043972f0729abf6019
4,0.866667,0.133333,12ff3219501d22c045a8ad0caaabb728
5,0.133333,0.866667,9a071773d2b55f6823530820584a3fba
6,0.433333,0.566667,981040a6959e0cf593c460210a820707
7,0.786667,0.213333,8ad553eacac5508054181c8b3f70c12b
8,0.066667,0.933333,4eda130f2da931d40253960e88ae5a33
9,0.250741,0.749259,a3f0c091a05bfb15a78bde8249660152


,educationInfoForAgeGroupEstimation,workInfoForAgeGroupEstimation,gender,realAge,ageGroup,relationship,educationLevel,occupation,income,topic_0,...,i1000,_id,ftopic_0,ftopic_1,ftopic_2,ftopic_3,ftopic_4,ftopic_5,lda50_female,lda50_male
0,Universidad Santo Tomï¿½s Class of 2009 ï¿½ Ps...,GMO Chile Analista de Selecciï¿½n y Desarrollo...,female,NaN,NaN,NaN,NaN,NaN,NaN,0.001808,...,NaN,062bbc1961e4b64f2c31b85d382b2384,0.188927,0.030995,0.034959,0.080224,0.586303,0.078591,0.700000,0.300000
1,NaN,NaN,male,NaN,NaN,NaN,NaN,NaN,NaN,0.000015,...,2.53352,aeb08521a61fc8e5900bc8dff0d4ba5b,0.062030,0.000452,0.219077,0.041921,0.012263,0.664257,0.100000,0.900000
2,NaN,NaN,male,NaN,NaN,in a relationship,NaN,NaN,NaN,0.001001,...,NaN,b22e8cc09aeb634daff8a0713dfd1808,0.798687,0.020560,0.023189,0.053214,0.052219,0.052131,0.266667,0.733333
3,NaN,Commonwealth of Massachusetts Legislative Aide...,male,NaN,NaN,NaN,NaN,healthcare support,NaN,0.001594,...,NaN,1a82db6eb37170043972f0729abf6019,0.696505,0.030995,0.034959,0.080224,0.078724,0.078591,0.700000,0.300000
4,NaN,Mindshare Pan Regional Director ï¿½ 1 October ...,female,NaN,NaN,NaN,NaN,management,NaN,0.001808,...,NaN,12ff3219501d22c045a8ad0caaabb728,0.188927,0.030995,0.034959,0.080224,0.078724,0.586170,0.866667,0.133333
5,West Milford Township High School West Milford...,Jackpocket CEO &amp; Founder ï¿½ May 2013 to p...,male,NaN,NaN,NaN,school,NaN,NaN,0.001808,...,NaN,9a071773d2b55f6823530820584a3fba,0.188927,0.538574,0.034959,0.080224,0.078724,0.078591,0.133333,0.866667
6,NaN,NaN,male,NaN,NaN,NaN,NaN,NaN,NaN,0.002089,...,NaN,981040a6959e0cf593c460210a820707,0.696505,0.030995,0.034959,0.080224,0.078724,0.078591,0.433333,0.566667
7,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,NaN,0.001425,...,NaN,8ad553eacac5508054181c8b3f70c12b,0.188927,0.030995,0.034959,0.334014,0.078724,0.332381,0.786667,0.213333
8,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,NaN,0.002473,...,NaN,4eda130f2da931d40253960e88ae5a33,0.696505,0.030995,0.034959,0.080224,0.078724,0.078591,0.066667,0.933333
9,NaN,Commonwealth Foundation Executive Director ï¿½...,male,NaN,NaN,NaN,NaN,management,NaN,0.001594,...,NaN,a3f0c091a05bfb15a78bde8249660152,0.188927,0.030995,0.034959,0.080224,0.078724,0.586170,0.250741,0.749259


,educationInfoForAgeGroupEstimation,workInfoForAgeGroupEstimation,gender,realAge,ageGroup,relationship,educationLevel,occupation,income,topic_0,...,i998,i999,i1000,_id,ftopic_0,ftopic_1,ftopic_2,ftopic_3,ftopic_4,ftopic_5
0,Universidad Santo Tomï¿½s Class of 2009 ï¿½ Ps...,GMO Chile Analista de Selecciï¿½n y Desarrollo...,female,NaN,NaN,NaN,NaN,NaN,NaN,0.001808,...,NaN,NaN,NaN,062bbc1961e4b64f2c31b85d382b2384,0.188927,0.030995,0.034959,0.080224,0.586303,0.078591
1,NaN,NaN,male,NaN,NaN,NaN,NaN,NaN,NaN,0.000015,...,2.600426,2.36398,2.53352,aeb08521a61fc8e5900bc8dff0d4ba5b,0.062030,0.000452,0.219077,0.041921,0.012263,0.664257
2,NaN,NaN,male,NaN,NaN,in a relationship,NaN,NaN,NaN,0.001001,...,NaN,NaN,NaN,b22e8cc09aeb634daff8a0713dfd1808,0.798687,0.020560,0.023189,0.053214,0.052219,0.052131
3,NaN,Commonwealth of Massachusetts Legislative Aide...,male,NaN,NaN,NaN,NaN,healthcare support,NaN,0.001594,...,NaN,NaN,NaN,1a82db6eb37170043972f0729abf6019,0.696505,0.030995,0.034959,0.080224,0.078724,0.078591
4,NaN,Mindshare Pan Regional Director ï¿½ 1 October ...,female,NaN,NaN,NaN,NaN,management,NaN,0.001808,...,NaN,NaN,NaN,12ff3219501d22c045a8ad0caaabb728,0.188927,0.030995,0.034959,0.080224,0.078724,0.586170
5,West Milford Township High School West Milford...,Jackpocket CEO &amp; Founder ï¿½ May 2013 to p...,male,NaN,NaN,NaN,school,NaN,NaN,0.001808,...,NaN,NaN,NaN,9a071773d2b55f6823530820584a3fba,0.188927,0.538574,0.034959,0.080224,0.078724,0.078591
6,NaN,NaN,male,NaN,NaN,NaN,NaN,NaN,NaN,0.002089,...,NaN,NaN,NaN,981040a6959e0cf593c460210a820707,0.696505,0.030995,0.034959,0.080224,0.078724,0.078591
7,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,NaN,0.001425,...,NaN,NaN,NaN,8ad553eacac5508054181c8b3f70c12b,0.188927,0.030995,0.034959,0.334014,0.078724,0.332381
8,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,NaN,0.002473,...,NaN,NaN,NaN,4eda130f2da931d40253960e88ae5a33,0.696505,0.030995,0.034959,0.080224,0.078724,0.078591
9,NaN,Commonwealth Foundation Executive Director ï¿½...,male,NaN,NaN,NaN,NaN,management,NaN,0.001594,...,NaN,NaN,NaN,a3f0c091a05bfb15a78bde8249660152,0.188927,0.030995,0.034959,0.080224,0.078724,0.586170


,lda50_female,lda50_male,_id
0,0.800000,0.200000,062bbc1961e4b64f2c31b85d382b2384
1,0.166667,0.833333,aeb08521a61fc8e5900bc8dff0d4ba5b
2,0.305556,0.694444,b22e8cc09aeb634daff8a0713dfd1808
3,0.166667,0.833333,1a82db6eb37170043972f0729abf6019
4,0.733333,0.266667,12ff3219501d22c045a8ad0caaabb728
5,0.566667,0.433333,9a071773d2b55f6823530820584a3fba
6,0.346667,0.653333,981040a6959e0cf593c460210a820707
7,0.400000,0.600000,8ad553eacac5508054181c8b3f70c12b
8,0.511433,0.488567,4eda130f2da931d40253960e88ae5a33
9,0.494444,0.505556,a3f0c091a05bfb15a78bde8249660152


,educationInfoForAgeGroupEstimation,workInfoForAgeGroupEstimation,gender,realAge,ageGroup,relationship,educationLevel,occupation,income,topic_0,...,i1000,_id,ftopic_0,ftopic_1,ftopic_2,ftopic_3,ftopic_4,ftopic_5,lda50_female,lda50_male
0,Universidad Santo Tomï¿½s Class of 2009 ï¿½ Ps...,GMO Chile Analista de Selecciï¿½n y Desarrollo...,female,NaN,NaN,NaN,NaN,NaN,NaN,0.001808,...,NaN,062bbc1961e4b64f2c31b85d382b2384,0.188927,0.030995,0.034959,0.080224,0.586303,0.078591,0.800000,0.200000
1,NaN,NaN,male,NaN,NaN,NaN,NaN,NaN,NaN,0.000015,...,2.53352,aeb08521a61fc8e5900bc8dff0d4ba5b,0.062030,0.000452,0.219077,0.041921,0.012263,0.664257,0.166667,0.833333
2,NaN,NaN,male,NaN,NaN,in a relationship,NaN,NaN,NaN,0.001001,...,NaN,b22e8cc09aeb634daff8a0713dfd1808,0.798687,0.020560,0.023189,0.053214,0.052219,0.052131,0.305556,0.694444
3,NaN,Commonwealth of Massachusetts Legislative Aide...,male,NaN,NaN,NaN,NaN,healthcare support,NaN,0.001594,...,NaN,1a82db6eb37170043972f0729abf6019,0.696505,0.030995,0.034959,0.080224,0.078724,0.078591,0.166667,0.833333
4,NaN,Mindshare Pan Regional Director ï¿½ 1 October ...,female,NaN,NaN,NaN,NaN,management,NaN,0.001808,...,NaN,12ff3219501d22c045a8ad0caaabb728,0.188927,0.030995,0.034959,0.080224,0.078724,0.586170,0.733333,0.266667
5,West Milford Township High School West Milford...,Jackpocket CEO &amp; Founder ï¿½ May 2013 to p...,male,NaN,NaN,NaN,school,NaN,NaN,0.001808,...,NaN,9a071773d2b55f6823530820584a3fba,0.188927,0.538574,0.034959,0.080224,0.078724,0.078591,0.566667,0.433333
6,NaN,NaN,male,NaN,NaN,NaN,NaN,NaN,NaN,0.002089,...,NaN,981040a6959e0cf593c460210a820707,0.696505,0.030995,0.034959,0.080224,0.078724,0.078591,0.346667,0.653333
7,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,NaN,0.001425,...,NaN,8ad553eacac5508054181c8b3f70c12b,0.188927,0.030995,0.034959,0.334014,0.078724,0.332381,0.400000,0.600000
8,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,NaN,0.002473,...,NaN,4eda130f2da931d40253960e88ae5a33,0.696505,0.030995,0.034959,0.080224,0.078724,0.078591,0.511433,0.488567
9,NaN,Commonwealth Foundation Executive Director ï¿½...,male,NaN,NaN,NaN,NaN,management,NaN,0.001594,...,NaN,a3f0c091a05bfb15a78bde8249660152,0.188927,0.030995,0.034959,0.080224,0.078724,0.586170,0.494444,0.505556
